<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=388234a7e4f443b5a558b43e93c9004814eb078c2c1ee3c8305de45eb9dacb7f
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-09 14:17:08
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.39 C
-------------------
Today PnL: 24.34 K (0.18%)
Current PnL: -29.15 L (-18.83%)
CY Booked + Current PnL: -14.86 L (-9.6%)
-------------------
Total profit:  1.15 L
Total loss:  -30.30 L
-------------------
Total Booked + Current PnL: 12.12 L (9.48%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.27%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 93.78 L (67.41%)
Deployed:  1.28 C
Current:  1.39 C
CAGR/XIRR %: 4.76%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-0.56,-18.11,22.22,0.09,18338.0,-18247.0,82530.0,91.46,31.0,M-SC,2.19,253.0,-1.00,0.62,8.43,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,0.88,-13.73,15.94,0.02,24003.0,-23973.0,150586.0,129.91,49.0,M-SC,16.69,234.0,-1.00,1.12,30.98,OX40N,NTT,PAINTS
43,ITC,452.00,0.06,-1.64,12.28,10.44,24173.0,-3291.0,196847.0,-41.15,45.0,X-LC,1.65,5.0,-0.14,1.47,3.83,X40,NTT,FMCG
50,MASFIN,398.61,1.30,-3.52,26.50,22.05,25050.0,-3450.0,94530.0,-16.89,53.0,H-SC,7.01,164.0,-0.14,0.70,37.41,XR,ATH,FINANCE
35,ICICIGI,2252.93,-0.86,3.72,15.88,20.19,28713.0,6491.0,180811.0,-18.61,40.0,X-MC,5.96,68.0,0.23,1.35,19.40,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,VALIANTORG,838.0,4.72,-43.27,206.96,74.15,192100.0,-70785.0,92820.0,-267.19,48.0,H-SC,21.14,149.0,-0.37,0.69,18.98,XR,NTT,CHEMICALS
59,RELAXO,1176.0,4.10,-46.36,183.96,52.31,143231.0,-67300.0,77860.0,-42.89,48.0,X-SC,5.97,91.0,-0.47,0.58,4.22,X40N,NTT,FOOTWEAR
10,BANDHANBNK,400.0,3.67,-32.44,184.82,92.42,347813.0,-90369.0,188190.0,32.18,33.0,H-SC,7.87,170.0,-0.26,1.40,8.76,XY24,NTT,BANKS
56,RAJESHEXPO,518.0,3.65,-63.73,175.93,0.07,87522.0,-87429.0,49748.0,1723.23,52.0,L-SC,2.36,268.0,-1.00,0.37,23.73,OX40N,NTT,JEWELLERY
44,JCHAC,2282.0,2.80,-37.15,59.10,-0.01,47470.0,-47484.0,80321.0,16334.78,17.0,M-SC,3.27,233.0,-1.00,0.60,2.80,OX40N,NTT,AC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ASIANPAINT,3460.25,-4.17,-1.91,23.30,20.95,57540.0,-4814.0,246954.0,-2.88,50.0,X-LC,19.04,31.0,-0.08,1.84,33.04,X40,ATH,PAINTS
2,ABBOTINDIA,35195.00,-2.16,-6.22,25.23,17.44,35454.0,-9322.0,140525.0,-19.22,32.0,X-MC,4.57,63.0,-0.26,1.05,11.74,X40,ATH,PHARMA
18,COFFEEDAY,80.00,-1.91,-47.07,155.59,35.27,93503.0,-53453.0,60096.0,-59.76,20.0,L-SC,12.39,270.0,-0.57,0.45,47.09,XR,NTT,HOTELS
28,GREENPANEL,537.00,-1.87,-39.15,135.22,43.14,150649.0,-71668.0,111410.0,149.35,22.0,M-SC,8.65,240.0,-0.48,0.83,4.80,XY24,NTT,MISC
49,LAOPALA,464.00,-1.85,-39.22,128.85,39.09,102669.0,-51424.0,79681.0,74.23,25.0,H-SC,7.36,155.0,-0.50,0.59,2.62,AR,NTT,CERAMICS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,1.66,-7.65,125.15,107.91,170632.0,-11302.0,136342.0,-25.01,33.0,M-SC,11.85,216.0,-0.07,1.02,1.66,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.88,-13.73,15.94,0.02,24003.0,-23973.0,150586.0,129.91,49.0,M-SC,16.69,234.0,-1.00,1.12,30.98,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.56,-18.11,22.22,0.09,18338.0,-18247.0,82530.0,91.46,31.0,M-SC,2.19,253.0,-1.00,0.62,8.43,OX40N,NTT,DURABLES
66,SIS,528.00,0.51,-24.80,61.25,21.26,51144.0,-27532.0,83500.0,1968.91,46.0,H-SC,4.60,166.0,-0.54,0.62,12.82,OX40N,NTT,MISC
47,KANSAINER,340.00,0.53,-25.56,52.45,13.47,105281.0,-68940.0,200727.0,-69.48,28.0,H-SC,3.19,158.0,-0.65,1.50,2.06,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,-1.34,5.77,62.87,72.27,112680.0,9779.0,179227.0,-9.26,65.0,M-LC,2.23,99.0,0.09,1.34,12.3,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-1.34,5.77,62.87,72.27,112680.0,9779.0,179227.0,-9.26,65.0,M-LC,2.23,99.0,0.09,1.34,12.30,XR,NTT,IT
38,INDIAMART,4810.62,-0.41,-2.71,112.47,106.72,134961.0,-3339.0,119997.0,-51.77,34.0,H-SC,1.19,139.0,-0.02,0.89,20.04,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,-0.39,-2.88,39.66,35.63,80720.0,-6044.0,203530.0,-16.74,35.0,H-MC,3.16,119.0,-0.07,1.52,13.31,AR,ATH,PHARMA
25,FINCABLES,1641.55,1.66,-7.65,125.15,107.91,170632.0,-11302.0,136342.0,-25.01,33.0,M-SC,11.85,216.0,-0.07,1.02,1.66,OX40N,ATH,CABLES
37,IEX,219.00,-0.74,-4.40,55.47,48.63,106029.0,-8807.0,191147.0,-36.48,47.0,H-SC,14.09,136.0,-0.08,1.42,6.64,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,1.98,-21.89,90.05,48.45,141036.0,-43886.0,156620.0,-27.60,21.0,X-MC,11.17,64.0,-0.31,1.17,1.98,X40N,ATH,FINANCE
22,DIXON,18931.72,-0.37,-13.22,40.10,21.58,48768.0,-18522.0,121617.0,-56.26,24.0,X-MC,5.46,56.0,-0.38,0.91,7.67,X40N,ATH,IT
75,TMPV,600.00,-0.59,-17.95,73.29,42.18,166217.0,-49616.0,226794.0,-26.74,26.0,X-LC,5.08,3.0,-0.30,1.69,0.00,XY24,NTT,AUTO
33,HINDUNILVR,2922.00,-0.25,-10.21,26.60,13.67,61395.0,-26253.0,230810.0,-16.94,29.0,X-LC,2.98,14.0,-0.43,1.72,8.96,XY25,NTT,FMCG
46,JSWINFRA,342.00,0.59,-10.89,29.30,15.23,52312.0,-21809.0,178538.0,-23.85,30.0,X-MC,7.89,66.0,-0.42,1.33,17.85,X40N,NTT,REALTY


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,485.0,-1.58,-21.94,96.63,53.50,227613.0,-66200.0,235551.0,-62.23,31.0,X-MC,1.25,58.0,-0.29,1.76,4.62,XY24,NTT,FMCG
43,ITC,452.0,0.06,-1.64,12.28,10.44,24173.0,-3291.0,196847.0,-41.15,45.0,X-LC,1.65,5.0,-0.14,1.47,3.83,X40,NTT,FMCG
3,ACC,3906.0,-0.58,-24.96,118.76,64.15,212046.0,-59401.0,178550.0,-56.31,36.0,X-SC,1.79,82.0,-0.28,1.33,0.16,XY24,BTT,CEMENT
15,CAMS,4762.0,-0.66,-80.39,536.29,24.79,264895.0,-202462.0,49394.0,-81.32,40.0,X-SC,1.84,86.0,-0.76,0.37,20.75,X40N,NTT,MISC
36,ICICIPRULI,790.0,1.34,3.53,26.50,30.97,48489.0,6240.0,182978.0,-20.80,56.0,X-MC,2.07,75.0,0.13,1.36,16.54,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,PAGEIND,51605.07,-0.05,-9.09,38.67,26.07,57564.0,-14880.0,148860.0,-33.02,30.0,X-MC,11.48,55.0,-0.26,1.11,0.00,X40,ATH,APPARELS
75,TMPV,600.00,-0.59,-17.95,73.29,42.18,166217.0,-49616.0,226794.0,-26.74,26.0,X-LC,5.08,3.0,-0.30,1.69,0.00,XY24,NTT,AUTO
54,PGHH,17907.65,-1.02,-5.92,42.17,33.76,79676.0,-11880.0,188940.0,-33.58,34.0,X-MC,5.10,57.0,-0.15,1.41,0.07,X40,ATH,FMCG
3,ACC,3906.00,-0.58,-24.96,118.76,64.15,212046.0,-59401.0,178550.0,-56.31,36.0,X-SC,1.79,82.0,-0.28,1.33,0.16,XY24,BTT,CEMENT
31,HAVELLS,2069.16,0.96,-9.03,45.63,32.48,136145.0,-29623.0,298368.0,-13.63,42.0,X-MC,5.17,67.0,-0.22,2.22,1.36,X40,ATH,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,1.06,-30.31,104.96,42.84,47552.0,-19701.0,45305.0,-54.27,40.0,X-SC,37.04,83.0,-0.41,0.34,2.50,XY24,NTT,MISC
15,CAMS,4762.00,-0.66,-80.39,536.29,24.79,264895.0,-202462.0,49394.0,-81.32,40.0,X-SC,1.84,86.0,-0.76,0.37,20.75,X40N,NTT,MISC
59,RELAXO,1176.00,4.10,-46.36,183.96,52.31,143231.0,-67300.0,77860.0,-42.89,48.0,X-SC,5.97,91.0,-0.47,0.58,4.22,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,1.36,-37.98,118.02,35.20,94175.0,-48874.0,79796.0,4.55,30.0,X-SC,15.26,92.0,-0.52,0.59,1.36,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-0.37,-13.22,40.10,21.58,48768.0,-18522.0,121617.0,-56.26,24.0,X-MC,5.46,56.0,-0.38,0.91,7.67,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-0.84,-12.80,36.78,19.28,110959.0,-44272.0,301684.0,-23.95,63.0,X-LC,5.77,1.0,-0.40,2.25,11.53,X40,ATH,IT
41,INFY,2275.00,-0.91,8.66,42.53,54.87,144598.0,27103.0,339991.0,-15.29,63.0,X-LC,3.38,2.0,0.19,2.53,17.66,X40,BTT,IT
75,TMPV,600.00,-0.59,-17.95,73.29,42.18,166217.0,-49616.0,226794.0,-26.74,26.0,X-LC,5.08,3.0,-0.30,1.69,0.00,XY24,NTT,AUTO
81,VBL,671.64,-0.11,-5.20,43.12,35.67,129107.0,-16429.0,299413.0,-16.39,51.0,X-LC,4.82,4.0,-0.13,2.23,7.93,X40N,ATH,FMCG
43,ITC,452.00,0.06,-1.64,12.28,10.44,24173.0,-3291.0,196847.0,-41.15,45.0,X-LC,1.65,5.0,-0.14,1.47,3.83,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.52,-37.56,112.83,32.89,54393.0,-28998.0,48208.0,-695.06,55.0,L-MC,6.17,259.0,-0.53,0.36,32.76,XR,NTT,BANKS
6,ASIANTILES,137.00,0.92,-18.71,123.49,81.67,94170.0,-17553.0,76257.0,6850.00,35.0,L-SC,14.93,271.0,-0.19,0.57,47.68,XR,NTT,CERAMICS
50,MASFIN,398.61,1.30,-3.52,26.50,22.05,25050.0,-3450.0,94530.0,-16.89,53.0,H-SC,7.01,164.0,-0.14,0.70,37.41,XR,ATH,FINANCE
13,BSOFT,831.70,0.56,-21.94,93.17,50.79,102291.0,-30863.0,109790.0,0.12,70.0,H-SC,5.84,151.0,-0.30,0.82,27.83,XR,ATH,IT
39,INDIGOPNTS,1408.00,0.88,-13.73,15.94,0.02,24003.0,-23973.0,150586.0,129.91,49.0,M-SC,16.69,234.0,-1.00,1.12,30.98,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,0.56,-21.94,93.17,50.79,102291.0,-30863.0,109790.0,0.12,70.0,H-SC,5.84,151.0,-0.30,0.82,27.83,XR,ATH,IT
32,HCLTECH,1922.01,-1.21,8.26,15.21,24.73,39836.0,19983.0,261907.0,13.18,65.0,X-LC,5.78,7.0,0.50,1.95,24.75,X40,ATH,IT
84,WIPRO,420.00,-1.34,5.77,62.87,72.27,112680.0,9779.0,179227.0,-9.26,65.0,M-LC,2.23,99.0,0.09,1.34,12.30,XR,NTT,IT
73,TCS,4389.97,-0.84,-12.80,36.78,19.28,110959.0,-44272.0,301684.0,-23.95,63.0,X-LC,5.77,1.0,-0.40,2.25,11.53,X40,ATH,IT
5,ASIANPAINT,3460.25,-4.17,-1.91,23.30,20.95,57540.0,-4814.0,246954.0,-2.88,50.0,X-LC,19.04,31.0,-0.08,1.84,33.04,X40,ATH,PAINTS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.78
1,25,45.76
2,50,76.72


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.98
MC    30.34
LC    24.71
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.30
X40      22.49
X40N     10.90
XR       10.05
XY25      9.41
AR        9.34
OX40N     7.77
SR        1.02
MH        0.75
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.51
X-MC    23.59
X-LC    21.14
M-SC    12.09
X-SC     6.99
H-MC     4.98
M-MC     1.41
L-SC     1.39
M-LC     1.34
H-LC     1.18
L-LC     1.05
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.43,-7.71,42.54
IT,12.85,-16.92,80.12
FINANCE,9.52,-21.37,72.65
ELECTRICAL,6.25,-11.91,52.89
MISC,6.09,-58.63,123.61
PAINTS,6.09,-13.04,29.83
INSURANCE,4.62,-1.97,37.91
PHARMA,4.09,-1.86,34.46
AUTO,3.42,-25.37,79.13


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3243285.0,21
AR,1318849.0,10
XR,1311839.0,13
X40,1020124.0,14
X40N,987343.0,9
OX40N,749905.0,10
XY25,380021.0,6
SR,283446.0,2
MH,83575.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3687654.0,25
M-SC,1437948.0,15
X-MC,1319958.0,16
X-SC,983084.0,8
X-LC,925678.0,11
H-MC,411180.0,3
L-SC,275195.0,3
M-LC,112680.0,1
H-LC,75499.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1293562.0      6
           AR         899696.0      5
M-SC       XY24       818266.0      6
H-SC       XR         781109.0      7
X-SC       X40N       504762.0      3
X-MC       X40        458886.0      7
           XY24       396739.0      3
X-LC       X40        377106.0      5
H-SC       OX40N      346266.0      4
M-SC       OX40N      316117.0      5
X-SC       XY24       294190.0      3
H-SC       SR         283446.0      2
X-MC       X40N       278397.0      4
X-LC       XY24       245421.0      2
H-MC       AR         216073.0      2
X-LC       X40N       204184.0      2
H-MC       XY24       195107.0      1
L-SC       XR         187673.0      2
X-MC       XY25       185936.0      2
X-SC       X40        184132.0      2
M-SC       XR         175984.0      2
           AR         127581.0      2
M-LC       XR         112680.0      1
X-LC       XY25        98967.0      2
L-SC       OX40N       87522.0      1
H-SC       MH          83575.0      1
H-LC       AR          75499.0      1
M-MC       XY25        56405.0      1
L-MC       XR          54393.0      1
L-LC       XY25        38713.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 25.0 seconds
